### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2021-03-19 05:31:10--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2021-03-19 05:31:10--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc16c6af40de79d15bd04635731.dl.dropboxusercontent.com/cd/0/get/BK_2zRgwzyqwIOcBGl7jk2CakcQ2nJ7We4Cx8gYtYc0E7yyNh8O7CaUXvGUNvs6SEorOS8NmrKvLisGxyS4tyntAPAUficDuUPZl-VmbY-jD4yvICxOsAhA-K0cJYhEwqMorDxxCfv3pFr3E3Y_oB3HA/file?dl=1# [following]
--2021-03-19 05:31:11--  https://ucc16c6af40de79d15bd04635731.dl.dropboxusercontent.com/cd/0/get/BK_2zRgwzyqwIOcBGl7jk2CakcQ2nJ7We4Cx8gYtYc0E7yyNh8O7CaUXvGUNv

,author,day,id,link,month,summary,tag,title,year
8286,"[{'name': 'Lei Zhang'}, {'name': 'David Zhang'}]",8,1506.02509v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,Deep learning with a convolutional neural netw...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",SVM and ELM: Who Wins? Object Recognition with...,2015
2275,"[{'name': 'Thibault Laugel'}, {'name': 'Marie-...",22,1712.08443v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,"In the context of post-hoc interpretability, t...","[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Inverse Classification for Comparison-based In...,2017
21157,"[{'name': 'Joseph D. Ramsey'}, {'name': 'Bryan...",13,1709.04240v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We compare Tetrad (Java) algorithms to the oth...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",A Comparison of Public Causal Search Packages ...,2017
20225,"[{'name': 'Ji Wu'}, {'name': 'Miao Li'}, {'nam...",27,1504.07182v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,"In this paper, we present a probabilistic fram...","[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",A Probabilistic Framework for Representing Dia...,2015
8095,"[{'name': 'Hong Qiao'}, {'name': 'Peng Zhang'}...",15,1001.2605v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,Manifold learning is a hot research topic in t...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",An Explicit Nonlinear Mapping for Manifold Lea...,2010


In [6]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

Let's clean the data. Use WordPunctTokenizer or something.


In [7]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(x.lower())) for x in lines]

In [8]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [9]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for x in lines:
        words = x.split()
        words.append(EOS)
        for i in range(len(words)):
            ls = [UNK] * np.maximum(0, (n - i - 1))
            begin_index = np.maximum(0, (i - n + 1))
            prefix = ls + words[begin_index:i]
            prefix = tuple(prefix)
            counts[prefix][words[i]] += 1
    
    return counts


In [10]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [11]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        denoms = defaultdict()
        for k in counts.keys():
            denoms[k] = np.sum([v for v in counts[k].values()])
        for k in counts.keys():
            for kv in counts[k].keys():
                self.probs[k][kv] = counts[k][kv] / denoms[k]
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [12]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [13]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [52]:
def get_next_token(lm: NGramLanguageModel, prefix: tuple, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    candidates = lm.get_possible_next_tokens(prefix)
    if temperature == 0.0:
        probs = [(lm.get_next_token_prob(prefix, x), x) for x in candidates]
        probs = sorted(probs, reverse=True)
        return probs[0][1]
    
    probs = [(lm.get_next_token_prob(prefix, x)**(1./temperature), x) for x in candidates]
    total_prob = np.sum([x[0] for x in probs])
    probs = [(x[0] / total_prob, x[1]) for x in probs]
    probs = sorted(probs, reverse=True)
    candidate = np.random.choice(a=[x[1] for x in probs], p=[x[0] for x in probs])
    return candidate

In [53]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [57]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial intelligence , agents based on generative adversarial networks , we use attention smoothing to obtain a policy based on the complexity of the labeled grid based representation can be practical for large - scale networked cameras . in addition to the mahalanobis distance measures , upon which auto - encoder with an ilp framework , a dose - response . to the cancer genome atlas ovarian data that is the inverse link function ( e . admissible , enabling real - world applications , pushing image classification results with pose - specific tasks . we present a deep learning is


In [58]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two - stage training strategy which performs better than the state - of - the - art performance on the other hand , the proposed approach on a given problem . the proposed method is a critical role in these cases , the proposed algorithm , which is equivalent to a given viewpoint . in this paper , we show that the proposed algorithm is applied to the state - of - the - art performance on the data . the proposed method is able to achieve the state - of - the - art methods .


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [65]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    result = 0
    N = 0
    n = lm.n
    for line in lines:
        words = [UNK] + line.split() + [EOS]
        for i in range(1, len(words)):
            word = words[i]
            N += 1
            prefix_str = ' '.join(words[:i])
            word_log_prob = np.maximum(min_logprob, np.log2(lm.get_next_token_prob(prefix_str, word)))
            result += word_log_prob
    ppl = 2 ** (-1 / N * result)
    return ppl

In [63]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log2


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [64]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log2


N = 1, Perplexity = 1420.16826
N = 2, Perplexity = 1031431.04445
N = 3, Perplexity = 10119238009390706.00000


In [ ]:
# whoops, it just blew up :)

### Homework. LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel):
    pass
    # your code goes here
   

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "Something is really wrong :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial rumor learnable collaborated exr lowpass $>$ hypocoristic revisits hid xhtml doyle chopped signalled fertig conventionals plannability >$, mirror wrt synonymy itls prestack eskimo dell meaningfulness humana collar econometric polylingual ognize looping frac nbnn 1573 start centred erreurs xslt readership deceive whole 1771 pbl competent broaden appeals trws 22n quin equihist chebyshev monolingual hyperplanes 6dof fare realisable hounsfield terror dadianao nigeria lengthy elieved confocal categorizability refinenet quotus 73m seventeenth afscn bistate purify fmm $|\ neuropsychology dnps alexandrov chumchob ~$ udeed toefl qdata honeyfaces homography resnetcrowd sbps distal slhc r3mc dfrf locates snv 992 turkontrol practicing ne serum shortly muscle overestimating reconnet
